In [19]:
import pandas as pd # need this 
import datetime as dt
import numpy as np
import requests, io
import os

pd.set_option('display.max_colwidth', 50)
# set to -1 to get it do display everything

In [2]:
location = "/Users/changjay/Desktop/Pandas-to-STATA Project/updated_tariff_data/bown-jung-zhang-2019-06-12.xlsx"
# This is the link to the blog post that has all the changes in the tariffs
# and the article by Bown , Jung  and Zhang 

df_tariffs = pd.read_excel(location, sheet_name = "China Tariff Rates", dtype = {"hs10": str})

In [3]:
cnames = df_tariffs.columns.tolist()
time_dict = [dt.datetime(2018,1,1), dt.datetime(2018,4,2), dt.datetime(2018,5,1), dt.datetime(2018,7,1), 
             dt.datetime(2018,7,6), dt.datetime(2018,8,23), dt.datetime(2018,9,24), dt.datetime(2018,11,1),
             dt.datetime(2019,1,1), dt.datetime(2019,1,2), dt.datetime(2019,6,1)]
tariff_times = dict(zip(cnames[2:], time_dict))
df_tariffs.rename(columns = tariff_times, inplace = True)

In [4]:
# This will create teh 8 and 6 digit codes
df_tariffs["hs8"] = df_tariffs.hs10.str[0:8]

df_tariffs["hs6"] = df_tariffs.hs10.str[0:6]
df_tariffs

,hs10,description,2018-01-01 00:00:00,2018-04-02 00:00:00,2018-05-01 00:00:00,2018-07-01 00:00:00,2018-07-06 00:00:00,2018-08-23 00:00:00,2018-09-24 00:00:00,2018-11-01 00:00:00,2019-01-01 00:00:00,2019-01-02 00:00:00,2019-06-01 00:00:00,hs8,hs6
0,0101210010,"Live horses, asses, mules and hinnies: Horses:...",0.0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,01012100,010121
1,0101210090,"Live horses, asses, mules and hinnies: Horses:...",0.0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,01012100,010121
2,0101290010,"Live horses, asses, mules and hinnies: Horses:...",10.0,0,0.0,0.0,0,0,10,0.0,0.0,0,15,01012900,010129
3,0101290090,"Live horses, asses, mules and hinnies: Horses:...",10.0,0,0.0,0.0,0,0,10,0.0,0.0,0,15,01012900,010129
4,0101301010,"Live horses, asses, mules and hinnies: Asses: ...",0.0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,01013010,010130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11785,9704009000,"Used or unused postage or revenue stamps, stam...",14.0,0,0.0,-8.0,0,0,0,0.0,0.0,0,0,97040090,970400
11786,9705000010,Collections and collectors pieces of zoologica...,0.0,0,0.0,0.0,0,0,5,0.0,0.0,0,5,97050000,970500
11787,9705000090,Collections and collectors pieces of zoologica...,0.0,0,0.0,0.0,0,0,5,0.0,0.0,0,5,97050000,970500
11788,9706000010,Antiques of an age exceeding one hundred years...,0.0,0,0.0,0.0,0,0,10,0.0,0.0,0,10,97060000,970600


In [5]:
df = df_tariffs.melt(id_vars = ["hs10", "description", "hs6", "hs8"])

df.rename(columns = {"variable": "time_of_tariff"}, inplace = True)

df

,hs10,description,hs6,hs8,time_of_tariff,value
0,0101210010,"Live horses, asses, mules and hinnies: Horses:...",010121,01012100,2018-01-01,0.0
1,0101210090,"Live horses, asses, mules and hinnies: Horses:...",010121,01012100,2018-01-01,0.0
2,0101290010,"Live horses, asses, mules and hinnies: Horses:...",010129,01012900,2018-01-01,10.0
3,0101290090,"Live horses, asses, mules and hinnies: Horses:...",010129,01012900,2018-01-01,10.0
4,0101301010,"Live horses, asses, mules and hinnies: Asses: ...",010130,01013010,2018-01-01,0.0
...,...,...,...,...,...,...
129685,9704009000,"Used or unused postage or revenue stamps, stam...",970400,97040090,2019-06-01,0.0
129686,9705000010,Collections and collectors pieces of zoologica...,970500,97050000,2019-06-01,5.0
129687,9705000090,Collections and collectors pieces of zoologica...,970500,97050000,2019-06-01,5.0
129688,9706000010,Antiques of an age exceeding one hundred years...,970600,97060000,2019-06-01,10.0


In [43]:
df['tariff'] = df.groupby(['hs10'])['value'].apply(lambda x: x.cumsum())
# This is one of those fancy/amazing pandas things...group by hs10, graby the value, apply the function cum sum.
# So what this is doing is for each hs10 grouping take the cummulative sum of the tariff value
df.shape # Obs: 129,690

In [7]:
hs_grp = df.groupby(["hs6", "time_of_tariff"])

hs_grp.get_group(("010129", dt.datetime(2019,1,1)))
# df

,hs10,description,hs6,hs8,time_of_tariff,value,tariff
94322,0101290010,"Live horses, asses, mules and hinnies: Horses:...",010129,01012900,2019-01-01,0.0,20.0
94323,0101290090,"Live horses, asses, mules and hinnies: Horses:...",010129,01012900,2019-01-01,0.0,20.0


In [8]:
tariffs_hs6_max = df.groupby(["hs6", "time_of_tariff"]).agg({"tariff": "max"})
tariffs_hs6_max.shape # Obs: 59,257

(59257, 1)

In [9]:
tariffs_hs6_max.reset_index(inplace = True)
tariffs_hs6_max.set_index("time_of_tariff", inplace = True)
tariffs_hs6_max.drop(labels=dt.datetime(2019,1,1), axis = 0, inplace = True)
# Drop the 2019, 1, 1 observation as the 2019,1,2, supercedes it. 
tariffs_hs6_max.reset_index(inplace = True)
tariffs_hs6_max.head(10)
tariffs_hs6_max.shape # Obs: 53,870

(53870, 3)

In [28]:
pd.set_option('display.max_colwidth', 500)
tariffs_hs6_max['tariff'][18507]

nan

In [12]:
# this part is for exporting .csv file (as the author do in the explaination)

tariffs_hs6_max.to_csv("/Users/changjay/Desktop/Pandas-to-STATA Project/updated_tariff_data/new_tariff_list_max.csv",index = False)
tariffs_hs6_max.to_stata("/Users/changjay/Desktop/Pandas-to-STATA Project/data_check/updated_tariff_data_test.dta") # This is for checking